In [1]:
import pandas as pd
import numpy as np

In [38]:
path="/raid/abhilash/updated_synthetic_datasets/ETTh2/p7/v0_mcar_etth2_imputed_SAITS.csv"
gt_path="/raid/abhilash/forecasting_datasets/ETT/ETTh2.csv"

In [39]:
df=pd.read_csv(path)
df_gt=pd.read_csv(gt_path)

In [24]:
df

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 00:00:00,41.130000,12.481000,36.580338,9.355000,3.802590,1.497448,29.394530
1,2016-07-01 01:00:00,37.528000,10.136000,32.197647,7.420216,4.435000,1.215000,31.396818
2,2016-07-01 02:00:00,36.126790,11.309000,35.330000,9.515354,2.100000,0.213972,32.751106
3,2016-07-01 03:00:00,36.223663,11.259777,33.382248,9.293159,2.918706,0.349026,31.813652
4,2016-07-01 04:00:00,38.114000,11.593906,35.160873,9.510247,2.306497,0.497732,31.850500
...,...,...,...,...,...,...,...,...
15067,2016-11-11 19:00:00,27.216808,6.642453,39.162487,9.420820,-10.310001,-2.637860,14.497912
15068,2016-11-11 20:00:00,25.732933,5.948000,36.311123,7.935000,-10.118673,-2.964000,14.900847
15069,2016-11-11 21:00:00,28.581146,5.223948,39.109510,8.098658,-9.048234,-2.097341,13.257886
15070,2016-11-11 22:00:00,31.078000,5.026000,41.087050,7.768622,-9.499826,-2.896227,13.618277


In [40]:
df_gt

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 00:00:00,41.130001,12.481,36.535999,9.355,4.424,1.311,38.661999
1,2016-07-01 01:00:00,37.528000,10.136,33.936001,7.532,4.435,1.215,37.124001
2,2016-07-01 02:00:00,37.946999,11.309,35.330002,9.007,2.100,0.000,36.465000
3,2016-07-01 03:00:00,38.952000,11.895,35.543999,9.436,3.380,1.215,33.608501
4,2016-07-01 04:00:00,38.113998,11.476,35.410000,9.623,2.036,0.000,31.850500
...,...,...,...,...,...,...,...,...
17415,2018-06-26 15:00:00,39.202999,11.392,49.644001,11.929,-10.331,-1.258,47.084999
17416,2018-06-26 16:00:00,38.113998,10.974,48.759998,11.366,-10.331,-1.290,48.183498
17417,2018-06-26 17:00:00,39.622002,10.974,50.609001,11.661,-11.557,-1.418,48.183498
17418,2018-06-26 18:00:00,43.643002,13.403,54.737000,13.778,-10.299,-1.418,46.865501


In [41]:
def get_train_val_test_split(path, dataset, flag):
    
    df_raw = pd.read_csv(path)
    seq_len=336
    if dataset=='ETTh2' or dataset=='ETTh1':
        border1s = [0, 12 * 30 * 24 - seq_len, 12 * 30 * 24 + 4 * 30 * 24 - seq_len]
        border2s = [12 * 30 * 24, 12 * 30 * 24 + 4 * 30 * 24, 12 * 30 * 24 + 8 * 30 * 24]
    elif dataset=='ETTm1' or dataset=='ETTm2':
        border1s = [0, 12 * 30 * 24 * 4 - seq_len, 12 * 30 * 24 * 4 + 4 * 30 * 24 * 4 - seq_len]
        border2s = [12 * 30 * 24 * 4, 12 * 30 * 24 * 4 + 4 * 30 * 24 * 4, 12 * 30 * 24 * 4 + 8 * 30 * 24 * 4]
    else:
        num_train = int(len(df_raw) * 0.7)
        num_test = int(len(df_raw) * 0.2)
        num_vali = len(df_raw) - num_train - num_test
        border1s = [0, num_train, len(df_raw) - num_test]
        border2s = [num_train, num_train + num_vali, len(df_raw)]
        
    border1 = border1s[flag]
    border2 = border2s[flag]

#     cols_data = df_raw.columns[1:]
#     date_col = df_raw.columns[:1]
#     df_data = df_raw[cols_data]
    
    data_split = df_raw[border1:border2]
    return data_split.reset_index(drop='true')

In [42]:
train_split=get_train_val_test_split(gt_path, 'ETTh2', 0)
val_split=get_train_val_test_split(gt_path, 'ETTh2', 1)
test_split=get_train_val_test_split(gt_path, 'ETTh2', 2)

In [43]:
train_split

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 00:00:00,41.130001,12.481,36.535999,9.355,4.424,1.311,38.661999
1,2016-07-01 01:00:00,37.528000,10.136,33.936001,7.532,4.435,1.215,37.124001
2,2016-07-01 02:00:00,37.946999,11.309,35.330002,9.007,2.100,0.000,36.465000
3,2016-07-01 03:00:00,38.952000,11.895,35.543999,9.436,3.380,1.215,33.608501
4,2016-07-01 04:00:00,38.113998,11.476,35.410000,9.623,2.036,0.000,31.850500
...,...,...,...,...,...,...,...,...
8635,2017-06-25 19:00:00,29.319000,5.696,38.118000,8.631,-8.337,-4.531,41.078999
8636,2017-06-25 20:00:00,31.832001,6.366,41.737000,9.543,-8.849,-3.262,39.760502
8637,2017-06-25 21:00:00,34.345001,8.293,43.130001,10.856,-8.316,-3.262,38.661999
8638,2017-06-25 22:00:00,33.172001,8.209,42.541000,11.232,-8.401,-3.262,37.563499


In [44]:
df_gt

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 00:00:00,41.130001,12.481,36.535999,9.355,4.424,1.311,38.661999
1,2016-07-01 01:00:00,37.528000,10.136,33.936001,7.532,4.435,1.215,37.124001
2,2016-07-01 02:00:00,37.946999,11.309,35.330002,9.007,2.100,0.000,36.465000
3,2016-07-01 03:00:00,38.952000,11.895,35.543999,9.436,3.380,1.215,33.608501
4,2016-07-01 04:00:00,38.113998,11.476,35.410000,9.623,2.036,0.000,31.850500
...,...,...,...,...,...,...,...,...
17415,2018-06-26 15:00:00,39.202999,11.392,49.644001,11.929,-10.331,-1.258,47.084999
17416,2018-06-26 16:00:00,38.113998,10.974,48.759998,11.366,-10.331,-1.290,48.183498
17417,2018-06-26 17:00:00,39.622002,10.974,50.609001,11.661,-11.557,-1.418,48.183498
17418,2018-06-26 18:00:00,43.643002,13.403,54.737000,13.778,-10.299,-1.418,46.865501


In [43]:
seq_len=336
mapper={0:'train',
       1:'val',
       2:'test'}

In [51]:
def get_train_val_test_split(path, dataset, flag):
    
    df_raw = pd.read_csv(path)

    if dataset=='ETTh2' or dataset=='ETTh1':
        border1s = [0, 12 * 30 * 24 - seq_len, 12 * 30 * 24 + 4 * 30 * 24 - seq_len]
        border2s = [12 * 30 * 24, 12 * 30 * 24 + 4 * 30 * 24, 12 * 30 * 24 + 8 * 30 * 24]
    elif dataset=='ETTm1' or dataset=='ETTm2':
        border1s = [0, 12 * 30 * 24 * 4 - seq_len, 12 * 30 * 24 * 4 + 4 * 30 * 24 * 4 - seq_len]
        border2s = [12 * 30 * 24 * 4, 12 * 30 * 24 * 4 + 4 * 30 * 24 * 4, 12 * 30 * 24 * 4 + 8 * 30 * 24 * 4]
    else:
        num_train = int(len(df_raw) * 0.7)
        num_test = int(len(df_raw) * 0.2)
        num_vali = len(df_raw) - num_train - num_test
        border1s = [0, num_train - seq_len, len(df_raw) - num_test - seq_len]
        border2s = [num_train, num_train + num_vali, len(df_raw)]
        
    border1 = border1s[flag]
    border2 = border2s[flag]

    cols_data = df_raw.columns[1:]
    df_data = df_raw[cols_data]
    
    data_split = df_data[border1:border2]
    
    '''
    save the split
    '''
    split_name = mapper[flag]
    data_split = data_split.reset_index(drop='true')
    
    save_path = '/'.join(path.split('/')[:-1]) + '/'
    filename = dataset + '_' + split_name + '.csv'
    data_split.to_csv(save_path+filename, index=0)
    return data_split

In [ ]:
etth2_path = "/raid/abhilash/forecasting_datasets/ETT/ETTh2.csv"
etth1_path = "/raid/abhilash/forecasting_datasets/ETT/ETTh1.csv"
ettm1_path = "/raid/abhilash/forecasting_datasets/ETT/ETTm1.csv"
ettm2_path = "/raid/abhilash/forecasting_datasets/ETT/ETTm2.csv"
weather_path = "/raid/abhilash/forecasting_datasets/weather/weather.csv"
ecl_path = "/raid/abhilash/forecasting_datasets/electricity/electricity.csv"
traffic_path = "/raid/abhilash/forecasting_datasets/traffic/traffic.csv"

In [57]:
datasets = {'ETTh1':etth1_path, 
            'ETTh2':etth2_path, 
            'ETTm1':ettm1_path, 
            'ETTm2':ettm2_path, 
            'weather':weather_path, 
            'electricity':ecl_path, 
            'traffic':traffic_path}

for dataset, path in datasets.items():
    print(f"Dataset :: {dataset}")
    train=get_train_val_test_split(path=path, dataset=dataset, flag=0)
    val=get_train_val_test_split(path=path, dataset=dataset, flag=1)
    test=get_train_val_test_split(path=path, dataset=dataset, flag=2)

In [60]:
!ls /raid/abhilash/forecasting_datasets/traffic/

traffic.csv  traffic_test.csv  traffic_train.csv  traffic_val.csv
